## Leitura dos dados em ROW

In [0]:
import requests

# URL to fetch the weather alerts
url = "https://alerts.weather.gov/cap/us.php?x=0"

# Fetch the data from the API
response = requests.get(url)
xml_data = response.content


In [0]:
import xml.etree.ElementTree as ET

# Parse the XML data
root = ET.fromstring(xml_data)


In [0]:
# Define a namespace dictionary to handle namespaces in the XML
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cap': 'urn:oasis:names:tc:emergency:cap:1.2'}

# Extract relevant data into a list of dictionaries
alerts = []
for entry in root.findall('atom:entry', namespaces):
    alert = {
        'id': entry.find('atom:id', namespaces).text,
        'updated': entry.find('atom:updated', namespaces).text,
        'published': entry.find('atom:published', namespaces).text,
        'title': entry.find('atom:title', namespaces).text,
        'summary': entry.find('atom:summary', namespaces).text,
        'event': entry.find('cap:event', namespaces).text if entry.find('cap:event', namespaces) is not None else None,
        'sent': entry.find('cap:sent', namespaces).text if entry.find('cap:sent', namespaces) is not None else None,
        'effective': entry.find('cap:effective', namespaces).text if entry.find('cap:effective', namespaces) is not None else None,
        'expires': entry.find('cap:expires', namespaces).text if entry.find('cap:expires', namespaces) is not None else None,
        'status': entry.find('cap:status', namespaces).text if entry.find('cap:status', namespaces) is not None else None,
        'msgType': entry.find('cap:msgType', namespaces).text if entry.find('cap:msgType', namespaces) is not None else None,
        'category': entry.find('cap:category', namespaces).text if entry.find('cap:category', namespaces) is not None else None,
        'urgency': entry.find('cap:urgency', namespaces).text if entry.find('cap:urgency', namespaces) is not None else None,
        'severity': entry.find('cap:severity', namespaces).text if entry.find('cap:severity', namespaces) is not None else None,
        'certainty': entry.find('cap:certainty', namespaces).text if entry.find('cap:certainty', namespaces) is not None else None,
        'areaDesc': entry.find('cap:areaDesc', namespaces).text if entry.find('cap:areaDesc', namespaces) is not None else None,
        'polygon': entry.find('cap:polygon', namespaces).text if entry.find('cap:polygon', namespaces) is not None else None,
        # Add more fields as needed
    }
    alerts.append(alert)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Weather Alerts with FIPS6 Mapping") \
    .getOrCreate()

# Define the schema for the DataFrame
schema = StructType([
    StructField("id", StringType(), True),
    StructField("updated", StringType(), True),
    StructField("published", StringType(), True),
    StructField("title", StringType(), True),
    StructField("summary", StringType(), True),
    StructField("event", StringType(), True),
    StructField("sent", StringType(), True),
    StructField("effective", StringType(), True),
    StructField("expires", StringType(), True),
    StructField("status", StringType(), True),
    StructField("msgType", StringType(), True),
    StructField("category", StringType(), True),
    StructField("urgency", StringType(), True),
    StructField("severity", StringType(), True),
    StructField("certainty", StringType(), True),
    StructField("areaDesc", StringType(), True),
    StructField("polygon", StringType(), True),
    # Add more fields as needed
])

# Convert the list of dictionaries to a DataFrame
alerts_df = spark.createDataFrame(alerts, schema)

# Show the DataFrame
alerts_df.show()


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-------+--------+---------+--------+---------+--------------------+--------------------+
|                  id|             updated|           published|               title|             summary|               event|                sent|           effective|             expires|status|msgType|category|  urgency|severity|certainty|            areaDesc|             polygon|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-------+--------+---------+--------+---------+--------------------+--------------------+
|https://api.weath...|2024-06-30T19:18:...|2024-06-30T19:18:...|Severe Thundersto...|SVRAKQ\n\nThe Nat...|Severe Thundersto...|2024-06-30T19:1

## Dataframe Pandas

In [0]:
# Convert the PySpark DataFrame to a pandas DataFrame
alerts_pandas_df = alerts_df.toPandas()
alerts_pandas_df



,id,updated,published,title,summary,event,sent,effective,expires,status,msgType,category,urgency,severity,certainty,areaDesc,polygon
0,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-30T19:18:00-04:00,2024-06-30T19:18:00-04:00,Severe Thunderstorm Warning issued June 30 at ...,SVRAKQ\n\nThe National Weather Service in Wake...,Severe Thunderstorm Warning,2024-06-30T19:18:00-04:00,2024-06-30T19:18:00-04:00,2024-06-30T19:45:00-04:00,Actual,Alert,Met,Immediate,Severe,Observed,"Caroline, VA; Hanover, VA","37.86,-77.59 37.95,-77.4599999 37.81,-77.33 37..."
1,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-30T19:17:00-04:00,2024-06-30T19:17:00-04:00,Severe Thunderstorm Warning issued June 30 at ...,"At 716 PM EDT, a severe thunderstorm was locat...",Severe Thunderstorm Warning,2024-06-30T19:17:00-04:00,2024-06-30T19:17:00-04:00,2024-06-30T19:30:00-04:00,Actual,Update,Met,Immediate,Severe,Observed,"Monmouth, NJ","40.13,-73.98 40.22,-74.29 40.25,-74.34 40.43,-..."
2,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-30T17:16:00-06:00,2024-06-30T17:16:00-06:00,Severe Thunderstorm Warning issued June 30 at ...,SVRBYZ\n\nThe National Weather Service in Bill...,Severe Thunderstorm Warning,2024-06-30T17:16:00-06:00,2024-06-30T17:16:00-06:00,2024-06-30T17:45:00-06:00,Actual,Alert,Met,Immediate,Severe,Observed,"Golden Valley, MT; Sweet Grass, MT; Wheatland, MT","45.99,-109.98 46.13,-110.2 46.61,-109.67 46.39..."
3,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-30T18:16:00-05:00,2024-06-30T18:16:00-05:00,Special Weather Statement issued June 30 at 6:...,"At 616 PM CDT, Doppler radar was tracking stro...",Special Weather Statement,2024-06-30T18:16:00-05:00,2024-06-30T18:16:00-05:00,2024-06-30T18:45:00-05:00,Actual,Alert,Met,Expected,Moderate,Observed,Richland; Madison; Franklin; Tensas; Issaquena...,"32.54,-91.26 32.54,-91.13 32.56,-91.11 32.56,-..."
4,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-30T19:14:00-04:00,2024-06-30T19:14:00-04:00,Severe Thunderstorm Warning issued June 30 at ...,SVRLWX\n\nThe National Weather Service in Ster...,Severe Thunderstorm Warning,2024-06-30T19:14:00-04:00,2024-06-30T19:14:00-04:00,2024-06-30T19:45:00-04:00,Actual,Alert,Met,Immediate,Severe,Observed,"Cecil, MD; Harford, MD","39.54,-76.04 39.54,-76.06 39.49,-76.11 39.45,-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-30T03:32:00-04:00,2024-06-30T03:32:00-04:00,Rip Current Statement issued June 30 at 3:32AM...,* WHAT...Dangerous rip currents.\n\n* WHERE......,Rip Current Statement,2024-06-30T03:32:00-04:00,2024-06-30T03:32:00-04:00,2024-06-30T11:45:00-04:00,Actual,Alert,Met,Expected,Moderate,Likely,Coastal Gulf,None
293,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-29T18:52:00-05:00,2024-06-29T18:52:00-05:00,Heat Advisory issued June 29 at 6:52PM CDT unt...,* WHAT...Heat index values up to 112 expected....,Heat Advisory,2024-06-29T18:52:00-05:00,2024-06-29T18:52:00-05:00,2024-06-30T03:00:00-05:00,Actual,Update,Met,Expected,Moderate,Likely,Williamson; Hays; Travis; Bastrop; Lee; Bexar;...,None
294,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-28T22:52:00-07:00,2024-06-28T22:52:00-07:00,Air Quality Alert issued June 28 at 10:52PM MS...,AQAPSR\n\nThe Arizona Department of Environmen...,Air Quality Alert,2024-06-28T22:52:00-07:00,2024-06-28T22:52:00-07:00,2024-06-30T21:00:00-07:00,Actual,Alert,Met,Unknown,Unknown,Unknown,"Maricopa, AZ",None
295,https://api.weather.gov/alerts/urn:oid:2.49.0....,2024-06-28T09:04:00-07:00,2024-06-28T09:04:00-07:00,Air Quality Alert issued June 28 at 9:04AM MST...,AQAPSR\n\nThe Arizona Department of Environmen...,Air Quality Alert,2024-06-28T09:04:00-07:00,2024-06-28T09:04:00-07:00,2024-06-30T21:00:00-07:00,Actual,Alert,Met,Unknown,Unknown,Unknown,"Maricopa, AZ",None


## GeoPandas


In [0]:
dbutils.library.installPyPI("geopandas")

## Camada Bronze Delta Table

In [0]:
# Camada Bronze - Raw Data
bronze_path = "/mnt/data/bronze"

# Salvar o DataFrame como uma tabela Delta
alerts_df.write.format("delta").mode("append").save(bronze_path)


## Camada Silver Delta Table

In [0]:
# Camada Silver - Cleaned and Transformed Data
silver_path = "/mnt/data/silver"

# Transformações, se necessário (exemplo simplificado)
silver_df = alerts_df.select("id", "updated", "published", "title", "event", "category")

# Salvar o DataFrame transformado como uma tabela Delta
silver_df.write.format("delta").mode("overwrite").save(silver_path)


## Camada Gold Delta Table

In [0]:
# Camada Gold - Enriched Data
gold_path = "/mnt/data/gold"

# Enriquecimento dos dados (exemplo simplificado)
gold_df = alerts_df.withColumn("full_title", alerts_df["title"] + " - " + alerts_df["event"])

# Salvar o DataFrame enriquecido como uma tabela Delta
gold_df.write.format("delta").mode("overwrite").save(gold_path)